# Scraping d'annonces immobilières sur ParuVendu

## Objectif du notebook

Ce notebook a pour objectif de collecter automatiquement des annonces immobilières en vente sur le site **ParuVendu**, pour plusieurs villes françaises.

Les données récupérées sont :
- le titre de l’annonce
- le lien
- la description
- le prix
- la localisation
- les détails du bien (surface, pièces, etc.)

Les résultats sont stockés dans un fichier CSV, avec un système de reprise automatique en cas d’arrêt du script (checkpoint).


## Choix du site ParuVendu

Le site ParuVendu a été retenu pour plusieurs raisons :

- il propose un grand nombre d’annonces immobilières
- les annonces sont accessibles sans authentification
- la structure des pages est relativement stable
- les informations clés sont présentes dès la page de liste


## Choix des villes françaises

Le scraping porte sur une sélection de **20 villes françaises**.

Ce choix vise à obtenir :
- une couverture géographique nationale
- une diversité de marchés immobiliers
- un équilibre entre grandes métropoles et villes moyennes

Les villes sélectionnées sont :

- Paris
- Marseille
- Lyon
- Toulouse
- Nice
- Nantes
- Montpellier
- Strasbourg
- Bordeaux
- Lille
- Rennes
- Reims
- Toulon
- Saint-Étienne
- Le Havre
- Grenoble
- Dijon
- Angers
- Nîmes
- Clermont-Ferrand


## Import des bibliothèques

Cette section permet d’importer l’ensemble des bibliothèques nécessaires au fonctionnement du scraping.

Chaque bibliothèque a un rôle spécifique :
- `requests` : envoi de requêtes HTTP vers le site
- `BeautifulSoup` : analyse et navigation dans le code HTML
- `csv` : lecture et écriture des données dans un fichier CSV
- `time` : gestion des pauses entre les requêtes
- `os` : gestion des chemins et des fichiers
- `json` : sauvegarde et lecture du checkpoint

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import os
import re
import json

## Configuration générale

Cette section regroupe tous les paramètres globaux du scraping :
- en-têtes HTTP
- villes ciblées
- nombre de pages
- limites de sécurité

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept-Language": "fr-FR,fr;q=0.9,en;q=0.8",
}

villes = [
    "paris-75", "marseille", "lyon", "toulouse", "nice",
    "nantes", "montpellier", "strasbourg", "bordeaux", "lille",
    "rennes", "reims", "toulon", "saint-etienne", "le-havre",
    "grenoble", "dijon", "angers", "nimes", "clermont-ferrand"
]

nb_pages = 5
url_base = "https://www.paruvendu.fr/immobilier/vente/"

MAX_ANNONCES_PAR_RUN = 1000
CHECKPOINT_FILE = "checkpoint.json"

# Ces paramètres permettent de contrôler précisément le volume de données collectées et de limiter les risques de blocage.